<a href="https://colab.research.google.com/github/SAPShikha/GargShikha.github.io/blob/main/EWA_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import string
# import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
stop_words=set(nltk.corpus.stopwords.words('english'))
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim


import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
# from spacy.tokens import Doc
# from spacy.tokenizer import Tokenizer
from spacy.lang.en import English


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_excel('SAP EarlyWatch Alert Solution Finder – Alert Worklist.xlsx')
df = df.drop(columns=['Since Date','Support Region',
       'Country/Region', 'Contract Type', 'SAP MaxAttention Only', 'HEC Only',
        'Link to SAP EarlyWatch Alert Viewer', 'Action', 'Assigned',
       'Status', 'Planned Closure date', 'Commentary'], axis=1)

df['Alert title'] = df['Alert/Title']
df['details'] = df['Detail Text']

df.rename(columns = {'Alert/Title' : 'text1',
                    'Detail Text' : 'text2'}, inplace = True)

df['Text'] = df['text1'] + " " + df['text2']

# df = df.dropna()
df = df[['Customer Number','Customer Name','Product','System ID',
         'System Number','Instance','Installation Number','Alert title','details',
         'Text']]
df = df.drop_duplicates()

In [4]:
rel_upg = "SAP will provide version upgrade management which includes Release upgrade, update planning, implementation and coordination as well as support for application Basis related tasks for the SAP cloud environment applications limited to adjustments in the SAP namespace following execution of SPDD, SPAU and SPAU_ENH. SAP will support Transport Management related upgrade activities, including Inform customer about scope of transports needed to be released. Provide general advice for issues related to transport problems. Provide general advice on creating transports to import in the upgrade process adjustments of repository objects as part of software changes SPDD, SPAU, SPAU_ENH of customer namespace. Define the sequence for the upgrade of tiers and systems. Plan communication and downtime alignment with business. Perform alignment, monitoring and controlling of Upgrade implementation including status calls. Run the Simplification Item and Apply corrections. Clean up transport management Release transports, import to all tiers, delete from import queue."
app_op = "SAP will provide support of application operations and continuous administrative support of systems in the SAP Cloud environment, including investigation and resolution of technical application related issues. SAP will perform changes to technical system configuration and service request fulfillment for application operations tasks. Number of Tickets. Provide detailed description of the interruption or service request and Analyze it and propose a solution to resolve the issue. Incident Management, Troubleshooting of technical/non-functional incidents. Changes of technical system configuration. Monitor technical/non-functional event types. Identify, analyze and optimize expensive SQL-statements to improve application performance. Create and modify users for HANA modelling in the SAP HANA Studio. User, roles, and permissions management for non-technical users. Administration of Agile Data Preparation (ADP). Set up replications, federation and transformations. Deploy standard adapters with SAP HANA. Monitor Data Provisioning tasks and remote subscriptions. Process remote subscription exceptions. Configure data source on Streaming Analytics option host. Create and deploy Streaming Analytics option streaming projects. Monitor Streaming Analytics option objects and projects. Process chain retriggering or cancellation. Generate, Renew, Extend Certificate Signing Request. Execute productive failover for a specific SID and invert replication vector additional customer requests. Regular ABAP dump check and classification. Analyze SAP application log and provide recommendations on fixing failures."
dqo = "SAP  profile data domains through routine scanning of data sources and provide data quality scores reflective of the data domain. SAP will initiate remediation activities for high risk findings. SAP will validate the setup and establish baseline configuration of Information Steward. SAP will identify the data sources for the respective data domains and establish baseline quality scores. SAP will scan data sources for the following Master Data Domains Customer Master Vendor Master Material Master Employee Master Financials Master Supplier Master SAP will provide the customer with a scorecard for data quality KPIs. SAP will identify areas of risk and initiate remediation for high risk findings. System landscape. Connect SAP Data Intelligence to the Customer data environment. Analyze applications, data policies, data lineage, integrity rules and cleansing success criteria. Collect metadata and assess data quality. Profile the data to assess quality. it makes sure that data is fit for consumption and meets the need of customers. addresses challenges faced by customers who need assistance with addressing issues related to poor data quality. Improve data quality. Data profiling and recommendations. Implement a data quality management solution. Monitor and manage data quality."
dvo = "SAP provide regular analysis of data volume and load, including forecast of growth trends. Analysis of data distribution among SAP components. forecast of future data volume growth. assistance with development of alternate plans such as archiving, consolidation and decommissioning. optimised data volume strategy. Deletion, avoiding, ageing, archiving. data growth exceeds original projections, business growth leads to unanticipated data growth , more data leads to higher costs. Manage disk capacity, volume and space."
asm = "SAP will continuously monitor Customer applications related to security statuses, so that the systems and SAP applications are operating in a secure mode. SAP reports Security KPIs and remediation activities that are monitored and performed on a quarterly basis. SAP will continuously monitor the Customer’s applications for security status and addresses all alerts or issues. SAP will review the security logs to initiate the adjustments of parameters of high risks. SAP will set up security monitoring and alerting in the Customer’s SAP Solution Manager. SAP will continuously monitor and report following Security KPI. Secure configuration of SAP system. Application Server security checks. Security Note Implementation Status. Profile Parameter. Security Audit Log settings. Critical authorization. Secure configuration of HANA system. Security settings. Critical privileges. SAP will initiate remediation activities to resolve identified risks. Customer will provide relevant system access. As prerequisite for setting up Security Monitoring there needs to be a Customer SAP Solution Manager in the system landscape where the basic setup is executed and the to be monitored systems need to be connected as managed systems."
reg = "SAP  will  perform  regression  testing  of the  SAP  S/4HANA cloud  environment  systems  in  scope as described herein following an update to the system. SAP will coordinate 1 regression test cycle annually, align with Customer’s test strategy, approach and schedule with respect to the Customer defined test scope. SAP will leverage Customer’s instance of Test Management Tool as system of record fortesting. SAP will provide configuration of Test Management Tool (e.g. Solution Manager, MicroFocus Quality Center, Tricentis, WorkSoft) SAP will provide maintenance of Test Management Tool during the regression testcycles. SAP will verify functionalities which are required for test planning, testing execution, including reporting of testing processes and defect creation within Customer’s Test Management Toolaccording to Customer’s test approach and make recommendations for configuration, ifrequired. SAP will use Customer provided test case documentation for test execution. SAP will perform the maintenance of test case documentation,  if required SAP will monitor and control testing execution, including reporting of testing progress to theCustomer. Identified defects will be reported and managed through the agreed Defect Management process"


stop_words = set(stopwords.words("english"))
stop_words.update(["sap","pdf","faq","cloud","system", "scope","launchpad","lifecycle","business",
                           "support","technical","pre","post","customer", "provide", 'perform', 'changes', 'per',"note",
                            "week","month","year","more","information",'hana','success','successful','usage', 'metric',
                           'following','including','include',"raci","matrix", "billion","note", "data"])

nlp = spacy.load("en_core_web_sm")

def summarize(doc,n):
  doc = nlp(doc)
  keyword = []
  pos_tag = ['PROPN','ADJ','NOUN','VERB']
  for token in doc:
    if(token.text in stop_words or token.text in punctuation):
      continue
    if(token.pos_ in pos_tag):
      keyword.append(token.text)
  freq_word = Counter(keyword)
  max_freq = Counter(keyword).most_common(1)[0][1]
  for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)

  sent_str = {}
  for sent in doc.sents:
    for word in sent:
      if word.text in freq_word.keys():
        if sent in sent_str.keys():
          sent_str[sent]+=freq_word[word.text]
        else:
          sent_str[sent]=freq_word[word.text]
  summ_sent = nlargest(n, sent_str, key = sent_str.get)
  return ' '.join([w.text for w in summ_sent])



rel_upg = summarize(rel_upg, 3)
app_op = summarize(app_op, 5)
dqo = summarize(dqo, 4)
dvo = dvo
asm = summarize(asm, 4)
reg = summarize(reg, 3)

In [5]:
def process_text(text):
    doc = nlp(str(text).lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        if token.is_digit:
            continue
        result.append(token.lemma_)
    return " ".join(result)

df['cleaned_text']=df['Text'].apply(process_text)

df['Release_Upgrade'] = rel_upg
df['Application_Operations'] = app_op
df['Data_Quality_Optimisation'] = dqo
df['Data_Volume_Optimization'] = dvo
df['Application_monitoring'] = asm
df['Regression_testing'] = reg

In [6]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=d750622feb6c5b000972a21e6dc08fd3096a175e54d01cc641ae8ef2e615f168
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [7]:
df.drop(columns = 'Text', inplace=True)
df2 = df.copy()

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def sentransformer_similarity(text1,text2):
    sentences = [text1, text2]
    embedding_1 = model.encode(sentences[0], convert_to_tensor=True)
    embedding_2 = model.encode(sentences[1], convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding_1, embedding_2)

In [8]:
df2['score1'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Release_Upgrade']).tolist()[0][0],2), axis=1)

In [9]:
df2['score2'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Application_Operations']).tolist()[0][0],2), axis=1)

In [10]:
df2['score3'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Data_Quality_Optimisation']).tolist()[0][0],2), axis=1)

In [11]:
df2['score4'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Data_Volume_Optimization']).tolist()[0][0],2), axis=1)

In [12]:
df2['score5'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Application_monitoring']).tolist()[0][0],2), axis=1)

In [13]:
df2['score6'] = df2.apply(lambda x: round(sentransformer_similarity(x['cleaned_text'],  x['Regression_testing']).tolist()[0][0],2), axis=1)

In [14]:
l = []
avg = []
for index,row in df2.iterrows():
  arr = [row['score1'], row['score2'],row['score3'], row['score4'], row['score5'], row['score6']]
  avg.append(round(sum(arr)/len(arr),2))
  l.append(max( (v, i) for i, v in enumerate(arr) )[1] + 1)
df2['Recommendation'] = l
df2['Avg_score'] = avg

df2['Recommendation'] = np.where(df2['Avg_score'] < 0.05, -1, df2['Recommendation'])

mapp = {1:'Release Version Upgrade',
        2:'Application Operations',
        3:'Data Quality Optimization',
        4:'Data Volume Optimization',
        5:'Application Monitoring',
        6:'Regression Testing',
        -1:'No Recommendation, Scores are too low'}

df2 = df2.replace({'Recommendation':mapp})
df2.head()

,Customer Number,Customer Name,Product,System ID,System Number,Instance,Installation Number,Alert title,details,cleaned_text,...,Application_monitoring,Regression_testing,score1,score2,score3,score4,score5,score6,Recommendation,Avg_score
0,1400626,Birkenstock Group B.V. & Co.KG,SAP S/4HANA (SAP S/4HANA 1809),SCP,500057160,SCP,21125679,To avoid loss of address data you need to impl...,Implement the SAP Notes for one-time addresses...,avoid loss address datum need implement sap ho...,...,SAP will continuously monitor Customer applica...,SAP will verify functionalities which are requ...,0.34,0.42,0.49,0.42,0.37,0.35,Data Quality Optimization,0.40
1,16830,PUMA SE,SAP ERP ENHANCE PACKAGE (EHP8 FOR SAP ERP 6.0),P02,500226747,P02,21208266,Based on response times in your ABAP system pe...,NaN,nan,...,SAP will continuously monitor Customer applica...,SAP will verify functionalities which are requ...,-0.03,0.06,0.04,0.03,0.00,-0.05,"No Recommendation, Scores are too low",0.01
2,16830,PUMA SE,SAP ERP ENHANCE PACKAGE (EHP8 FOR SAP ERP 6.0),PUP,500232707,PUP,21203481,Based on response times in your ABAP system pe...,NaN,nan,...,SAP will continuously monitor Customer applica...,SAP will verify functionalities which are requ...,-0.03,0.06,0.04,0.03,0.00,-0.05,"No Recommendation, Scores are too low",0.01
3,25353,Zurich Insurance Company Ltd.,SAP ERP ENHANCE PACKAGE (EHP8 FOR SAP ERP 6.0),T04,500336242,T04,21239000,Based on response times in your ABAP system pe...,NaN,nan,...,SAP will continuously monitor Customer applica...,SAP will verify functionalities which are requ...,-0.03,0.06,0.04,0.03,0.00,-0.05,"No Recommendation, Scores are too low",0.01
4,28556,Levi Strauss & Co.,SAP S/4HANA (SAP S/4HANA 2021),SGP,500182545,SGP,21123709,Based on response times in your ABAP system pe...,NaN,nan,...,SAP will continuously monitor Customer applica...,SAP will verify functionalities which are requ...,-0.03,0.06,0.04,0.03,0.00,-0.05,"No Recommendation, Scores are too low",0.01


In [15]:
df2['row_number'] = df2.reset_index().index
df2 = df2[['Customer Number', 'Customer Name', 'Product', 'System ID',
       'System Number','Alert title','details', 'Recommendation']]


In [16]:
df2.to_csv("EWA_CAS_Recommendation.csv")

In [17]:

!pwd

/content
